In [13]:
import spacy
from textblob import TextBlob
from spacy.tokens import Doc, Token
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
lyrics = """I hurt myself today
To see if I still feel
I focus on the pain
The only thing that's real
The needle tears a hole
The old familiar sting
Try to kill it all away
But I remember everything

What have I become?
My sweetest friend
Everyone I know
Goes away in the end
You could have it all
My empire of dirt
I will let you down
I will make you hurt

I wear this crown of shit
Upon my liar's chair
Full of broken thoughts
I cannot repair
Beneath the stains of time
The feelings disappear
You are someone else
I am still right here

What have I become?
My sweetest friend
Everyone I know
Goes away in the end

You could have it all
My empire of dirt
I will let you down
I will make you hurt
If I could start again
A million miles away
I would keep myself
I would find a way"""

In [6]:
Doc.set_extension('vader_negative', default=None, force=True)
Doc.set_extension('vader_neutral', default=None, force=True)
Doc.set_extension('vader_positive', default=None, force=True)
Token.set_extension('vader_negative', default=None, force=True)
Token.set_extension('vader_neutral', default=None, force=True)
Token.set_extension('vader_positive', default=None, force=True)

In [18]:
Doc.set_extension('tb_subjectivity', default=None, force=True)
Doc.set_extension('tb_polarity', default=None, force=True)

In [7]:
def vader_pipe(doc):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(doc.text)
    
    doc._.vader_negative = sentiment_dict['neg']
    doc._.vader_neutral = sentiment_dict['neu']
    doc._.vader_positive = sentiment_dict['pos']
    
    for sentence in doc.sents:
        for token in sentence:
            sentiment_dict_token = sid_obj.polarity_scores(token.text)
            token._.vader_negative = sentiment_dict_token['neg']
            token._.vader_neutral = sentiment_dict_token['neu']
            token._.vader_positive = sentiment_dict_token['pos']

    return doc

In [26]:
def textblob_pipe(doc):
    textBlobText = TextBlob(doc.text)
    
    doc._.tb_polarity = textBlobText.sentiment[0]
    doc._.tb_subjectivity = textBlobText.sentiment[1]

    return doc

In [27]:
if nlp.has_pipe('vader_pipe'):
    nlp.remove_pipe('vader_pipe')
nlp.add_pipe(vader_pipe)

In [28]:
if nlp.has_pipe('textblob_pipe'):
    nlp.remove_pipe('textblob_pipe')
nlp.add_pipe(textblob_pipe)

In [29]:
document = nlp(lyrics)
print(document._.vader_positive)
print(document._.vader_neutral)
print(document._.vader_negative)
print(document._.tb_polarity)
print(document._.tb_subjectivity)

0.051
0.738
0.211
0.03996031746031746
0.4863492063492063


In [10]:
text = nlp('John has cancer.')
print(text._.vader_positive)
print(text._.vader_neutral)
print(text._.vader_negative)

0.0
0.312
0.688


In [11]:
for sentence in text.sents:
    for token in sentence:
        if token.is_alpha:
            print(token)
            print(token._.vader_positive)
            print(token._.vader_neutral)
            print(token._.vader_negative)

John
0.0
1.0
0.0
has
0.0
1.0
0.0
cancer
0.0
0.0
1.0


In [15]:
textBlobText = TextBlob(lyrics)


In [25]:
textBlobText.sentiment

Sentiment(polarity=0.03996031746031746, subjectivity=0.4863492063492063)